# Modified 3D PMRNN

## Module Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import cv2
import numpy as np
import matplotlib.pyplot as plt
from CONFIGURATION import CONFIGURATION
from Reconstruction import Reconstruction

In [2]:
# ignorable
print("TensorFlow version:", tf.__version__)
print("cv2 version:",cv2.__version__)
print("np version:",np.__version__)

TensorFlow version: 2.7.0
cv2 version: 4.5.5
np version: 1.19.5


In [3]:
# ignorable
import sklearn, matplotlib
print("matplotlib version:",matplotlib.__version__)
print("sklearn version:",sklearn.__version__)

matplotlib version: 3.6.3
sklearn version: 1.2.2


In [4]:
from tensorflow.keras import layers,losses #,Sequential,metrics
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import * 
# from tensorflow.keras.layers import Layer
# from tensorflow.keras import optimizers 
# from tensorflow.keras.optimizers.experimental import SGD
# from tensorflow.image import ssim

## Load new Data

### reading data

In [ ]:
import Utils
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape

### Voxel extraction 

In [ ]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

In [ ]:
del image_data

### Train and test data extraction

In [ ]:
X_train = voxels[:-6]
X_test = voxels[-6:]

In [ ]:
del voxels

In [ ]:
len(X_train)

### Save Traited data for uses

In [ ]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [ ]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

## TensorFlow GPU setup

In [5]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
device = tf.config.list_physical_devices("GPU")
print(device)  # Check if GPU devices are visible
print(tf.test.is_built_with_cuda())  # Check if TensorFlow is built with CUDA support
print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True
(7, 2, 2)


In [ ]:
# only on weak GPU
tf.config.experimental.set_memory_growth(device[0],True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [6]:
tf.keras.backend.clear_session()

## Load Traited data for training (voxels)

In [7]:
X_train=np.load('X_train.npy')
X_train.shape

(58, 250, 256, 256, 1)

In [8]:
# for test only
X_train=X_train[:1]
with open('X_train_test.npy', 'wb') as f:
    np.save(f, X_train)
X_train.shape

(1, 250, 256, 256, 1)

In [ ]:
X_train=np.load('X_train_test.npy')
X_train.shape

(58, 250, 256, 256, 1)

### idk

In [9]:
learnedVoxels = []
inferenceVoxels = []
for voxel in X_train:
    learnedVoxels.append(voxel[:-1])
    inferenceVoxels.append(voxel[1:])

# learnedVoxels = np.array(learnedVoxels)
# inferenceVoxels = np.array(inferenceVoxels)

## Training 

### Imports

In [10]:
from mealpy.swarm_based import GWO
from sklearn.preprocessing import LabelEncoder
import time
import pickle
from datetime import datetime

### Lables

In [2]:
class Timer():
    def __init__(self,title=" ") -> None:
        self.title=title
        self.started=time.time()
        print(f"{self.title} Started at : {datetime.fromtimestamp(self.started)}")
        self.ended=None
    def End(self):
        self.ended=time.time()
        print(f"{self.title} Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s")
    def toDuration(self):
        if self.ended == None:
            return -1
        return self.ended-self.started
    def __str__(self) -> str:
        return f"{self.title} Started at : {datetime.fromtimestamp(self.started)} | Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s"

class TrainingData(Timer):
    def __init__(self,title=" ",p={}) -> None:
        super().__init__(title=title)
        self.samples=[]
        self.params=p
    
    def SampleStarted(self,title=" ")->Timer:
        timer= Timer(title)
        self.samples.append(timer)
        return timer
    
    def setParams(self,name,data):
        self.params[name]=data
    
    def __str__(self) -> str:
        return super().__str__()+" "+str(self.params)

In [12]:
OPT_ENCODER = LabelEncoder()
#OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'])
OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adam'])
trainings_duration=[]


### Solution Decodation function

In [13]:
def decode_solution(solution):
    batch_size = 2**int(solution[0])
    
    learning_rate = solution[1]
    
    opt_int = int(solution[2])
    opt = OPT_ENCODER.inverse_transform([opt_int])[0]
    
    epoch = 10 *int(solution[3])
    
    num_filters = int(solution[4])
    
    latent_space_dim = 2**int(solution[5])
    
    reduced_dim = 2**int(solution[6])
    
    return [batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim,reduced_dim]

### Objective Function

In [14]:
def objective_function(solution):
    tf.keras.backend.clear_session()
    batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(solution)
    reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                     latent_space_dim = latent_space_dim,
                                     reducedDimension = reduced_dim,
                                     num_conv_layers = num_filters,
                                     learning_rate = learning_rate,
                                     batch_size = batch_size,
                                     epochs = epoch,
                                     opt = opt,
                                   )
#     reconstruction.summary()
    reconstruction.compile()
    trainingTime = TrainingData("Epoch Training", {
        "inputShape": CONFIGURATION["INPUT_SHAPE"],
        "latent_space_dim": latent_space_dim,
        "reducedDimension": reduced_dim,
        "num_conv_layers": num_filters,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epoch,
        "opt": opt,
    })
    histories = []
    for i in range(len(learnedVoxels)):
        print(f"currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        sampleTime = trainingTime.SampleStarted(f"Voxel Training {i}")
        histories.append(reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))
        sampleTime.End()
        
    trainingTime.End()
    trainings_duration.append(trainingTime)
    reconstruction.save(save_folder=f"results/model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}")
    
    return histories[-1].history['loss'][-1]

In [ ]:
import random

# Define the range of float values
start = 0.001
end = 0.8


def objective_function(solution):
    tf.keras.backend.clear_session()
    batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = decode_solution(
        solution)
    """  reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                    latent_space_dim=latent_space_dim,
                                    reducedDimension=reduced_dim,
                                    num_conv_layers=num_filters,
                                    learning_rate=learning_rate,
                                    batch_size=batch_size,
                                    epochs=epoch,
                                    opt=opt,
                                    )
#     reconstruction.summary()
    reconstruction.compile() """

    trainingTime = TrainingData("Epoch Training", {
        "inputShape": CONFIGURATION["INPUT_SHAPE"],
        "latent_space_dim": latent_space_dim,
        "reducedDimension": reduced_dim,
        "num_conv_layers": num_filters,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epoch,
        "opt": opt,
    })
    histories = []
    for i in range(len(learnedVoxels)):
        sampleTime = trainingTime.SampleStarted(f"Voxel Training {i+1}")
        print(f"currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        # histories.append(reconstruction.train(learnedVoxels[i], inferenceVoxels[i]))
        sampleTime.End()

    trainingTime.End()
    trainings_duration.append(trainingTime)
    """ reconstruction.save(
        save_folder=f"results/model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}") """
    return random.uniform(start, end)
    #return histories[-1].history['loss'][-1]

### Configurations

In [15]:
LB = [3    , 0.001 , 0   , 2      ,  4  ,  6    , 7 ]
UB = [5.99 , 0.01  , 3.99, 4.99  ,  8.99,  8.99 , 9.99 ]

problem_dict = {
    "fit_func": objective_function,
    "lb": LB,
    "ub": UB,
    "minmax": "min",
    "verbose":True,
}
metaheuristic_configuration={
    "epoch":5,
    "pop_size":50
}
#save session for x epoch of the training
save_interval=5

### Lunching the metaheuristic

In [18]:
import pickle

# Check if a session file exists
session_file = 'gwo_session.pkl'
if os.path.exists(session_file):
    # Load the session and resume from the saved state
    with open(session_file, 'rb') as f:
        session_data = pickle.load(f)
    model = session_data['model']
    best_solution = session_data['best_solution']
    current_iteration = session_data['current_iteration']
    print("Resuming from iteration", current_iteration)
else:
    # Create a new GWO optimization session
    model = GWO.BaseGWO(problem_dict, epoch=metaheuristic_configuration["epoch"], pop_size=metaheuristic_configuration["pop_size"])
    best_solution = None
    current_iteration = 0

# Run the optimization loop
while current_iteration < metaheuristic_configuration["epoch"]:
    # Perform GWO iteration
    model.solve()

    # Update the best solution
    if best_solution is None or model.best_fit < best_solution["fit"]:
        best_solution = {
            "fit": model.best_fit,
            "position": model.best_solution
        }

    # Increment the iteration count
    current_iteration += 1

    # Save the session periodically
    if current_iteration % save_interval == 0:
        session_data = {
            "model": model,
            "best_solution": best_solution,
            "current_iteration": current_iteration
        }
        with open(session_file, 'wb') as f:
            pickle.dump(session_data, f)

# Delete the session file after the optimization is complete
if os.path.exists(session_file):
    os.remove(session_file)

Epoch Training Started at : 2023-06-18 11:42:56.786037
currently working one voxel : 1, voxels left : 0 
Voxel Training 0 Started at : 2023-06-18 11:42:56.786144
Train on 249 samples
Epoch 1/40


2023-06-18 11:43:08.316510: W tensorflow/core/common_runtime/bfc_allocator.cc:343] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


249/249 [==============================] - 11s 46ms/sample - loss: 0.1310 - _calculate_reconstruction_loss: 0.0392 - _calculate_porosity_loss: 0.0226 - _calculate_kl_loss: 66.5609
Epoch 2/40
249/249 [==============================] - 5s 19ms/sample - loss: 0.0894 - _calculate_reconstruction_loss: 0.0289 - _calculate_porosity_loss: 0.0042 - _calculate_kl_loss: 55.0290
Epoch 3/40
249/249 [==============================] - 4s 18ms/sample - loss: 0.0735 - _calculate_reconstruction_loss: 0.0281 - _calculate_porosity_loss: 0.0038 - _calculate_kl_loss: 40.5455
Epoch 4/40
249/249 [==============================] - 5s 18ms/sample - loss: 0.0581 - _calculate_reconstruction_loss: 0.0269 - _calculate_porosity_loss: 0.0042 - _calculate_kl_loss: 26.2009
Epoch 5/40
249/249 [==============================] - 5s 18ms/sample - loss: 0.0481 - _calculate_reconstruction_loss: 0.0259 - _calculate_porosity_loss: 0.0042 - _calculate_kl_loss: 17.4387
Epoch 6/40
249/249 [==============================] - 5s 18m

2023/06/18 11:46:37 AM, INFO, mealpy.swarm_based.GWO.BaseGWO: Solving single objective optimization problem.
2023/06/18 11:46:37 AM, INFO, mealpy.swarm_based.GWO.BaseGWO: Solving single objective optimization problem.
INFO:mealpy.swarm_based.GWO.BaseGWO:Solving single objective optimization problem.


Epoch Training Started at : 2023-06-18 11:46:41.922661
currently working one voxel : 1, voxels left : 0 
Voxel Training 0 Started at : 2023-06-18 11:46:41.922755
Train on 249 samples
Epoch 1/30
249/249 [==============================] - 9s 35ms/sample - loss: 0.7028 - _calculate_reconstruction_loss: 0.3561 - _calculate_porosity_loss: 0.2502 - _calculate_kl_loss: 92.6274
Epoch 2/30
249/249 [==============================] - 5s 20ms/sample - loss: 0.5826 - _calculate_reconstruction_loss: 0.2549 - _calculate_porosity_loss: 0.2320 - _calculate_kl_loss: 92.6551
Epoch 3/30
249/249 [==============================] - 6s 23ms/sample - loss: 0.5029 - _calculate_reconstruction_loss: 0.1933 - _calculate_porosity_loss: 0.2155 - _calculate_kl_loss: 91.0289
Epoch 4/30
249/249 [==============================] - 5s 21ms/sample - loss: 0.4477 - _calculate_reconstruction_loss: 0.1540 - _calculate_porosity_loss: 0.1980 - _calculate_kl_loss: 91.4063
Epoch 5/30
249/249 [==============================] - 5s 

...

Epoch Training Started at : 2023-06-18 15:07:21.419991
currently working one voxel : 1, voxels left : 0 
Voxel Training 0 Started at : 2023-06-18 15:07:21.420113
Train on 249 samples
Epoch 1/20
249/249 [==============================] - 9s 37ms/sample - loss: 0.1808 - _calculate_reconstruction_loss: 0.0710 - _calculate_porosity_loss: 0.0801 - _calculate_kl_loss: 26.1949
Epoch 2/20
249/249 [==============================] - 5s 21ms/sample - loss: 0.0582 - _calculate_reconstruction_loss: 0.0302 - _calculate_porosity_loss: 0.0051 - _calculate_kl_loss: 22.5403
Epoch 3/20
249/249 [==============================] - 5s 21ms/sample - loss: 0.0533 - _calculate_reconstruction_loss: 0.0275 - _calculate_porosity_loss: 0.0047 - _calculate_kl_loss: 20.8053
Epoch 4/20
249/249 [==============================] - 6s 23ms/sample - loss: 0.0520 - _calculate_reconstruction_loss: 0.0271 - _calculate_porosity_loss: 0.0042 - _calculate_kl_loss: 20.3449
Epoch 5/20
249/249 [==============================] - 7s 

Epoch Training Started at : 2023-06-18 15:09:31.115485
currently working one voxel : 1, voxels left : 0 
Voxel Training 0 Started at : 2023-06-18 15:09:31.115614
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 9s 35ms/sample - loss: 0.6609 - _calculate_reconstruction_loss: 0.3045 - _calculate_porosity_loss: 0.2403 - _calculate_kl_loss: 112.9973
Epoch 2/40
249/249 [==============================] - 2s 9ms/sample - loss: 0.4673 - _calculate_reconstruction_loss: 0.1547 - _calculate_porosity_loss: 0.1983 - _calculate_kl_loss: 113.2122
Epoch 3/40
249/249 [==============================] - 2s 9ms/sample - loss: 0.3823 - _calculate_reconstruction_loss: 0.1039 - _calculate_porosity_loss: 0.1653 - _calculate_kl_loss: 112.6078
Epoch 4/40
249/249 [==============================] - 2s 9ms/sample - loss: 0.3308 - _calculate_reconstruction_loss: 0.0797 - _calculate_porosity_loss: 0.1372 - _calculate_kl_loss: 113.1791
Epoch 5/40
249/249 [==============================] - 2s

Epoch Training Started at : 2023-06-18 15:11:44.571001
currently working one voxel : 1, voxels left : 0 
Voxel Training 0 Started at : 2023-06-18 15:11:44.571135
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 11s 43ms/sample - loss: 0.2329 - _calculate_reconstruction_loss: 0.0841 - _calculate_porosity_loss: 0.0594 - _calculate_kl_loss: 85.7606
Epoch 2/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.0757 - _calculate_reconstruction_loss: 0.0280 - _calculate_porosity_loss: 0.0054 - _calculate_kl_loss: 42.1888
Epoch 3/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.0610 - _calculate_reconstruction_loss: 0.0277 - _calculate_porosity_loss: 0.0048 - _calculate_kl_loss: 28.1918
Epoch 4/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.0723 - _calculate_reconstruction_loss: 0.0274 - _calculate_porosity_loss: 0.0046 - _calculate_kl_loss: 39.8940
Epoch 5/40


2023-06-18 15:12:27.390813: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 128.00MiB (rounded to 134217728)requested by op training/RMSprop/gradients/gradients/AddN_2-0-TransposeNHWCToNCHW-LayoutOptimizer
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-06-18 15:12:27.394952: W tensorflow/core/common_runtime/bfc_allocator.cc:474] ********x******************************x***********************x****x********_*************x*******x
2023-06-18 15:12:27.397305: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at transpose_op.cc:183 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[32,16,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc


ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[32,16,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/RMSprop/gradients/gradients/AddN_2-0-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[training/RMSprop/gradients/gradients/generate/batch_normalization_20/cond_grad/StatelessIf/then/_4262/gradients/FusedBatchNormV3_grad/FusedBatchNormGradV3/_3717]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[32,16,256,256] and type float on /job:localhost/replic

In [19]:
print(len(trainings_duration))
print(f"Best solution : {model.solution[0]}")
batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(model.solution[0])
print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}")
model.history.save_global_objectives_chart(filename="hello/goc")
model.history.save_local_objectives_chart(filename="hello/loc")

model.history.save_global_best_fitness_chart(filename="hello/gbfc")
model.history.save_local_best_fitness_chart(filename="hello/lbfc")

model.history.save_runtime_chart(filename="hello/rtc")

model.history.save_exploration_exploitation_chart(filename="hello/eec")

model.history.save_diversity_chart(filename="hello/dc")

model.history.save_trajectory_chart(list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello/tc")

72


TypeError: 'NoneType' object is not subscriptable

In [21]:
for training in trainings_duration:
    print(training)

Epoch Training Started at : 2023-06-18 11:38:28.350539 | Ended at : 2023-06-18 11:41:14.630616 and it took 166.28007626533508s
Epoch Training Started at : 2023-06-18 11:42:56.786037 | Ended at : 2023-06-18 11:46:36.423938 and it took 219.63790082931519s
Epoch Training Started at : 2023-06-18 11:46:41.922661 | Ended at : 2023-06-18 11:49:31.296244 and it took 169.37358260154724s
Epoch Training Started at : 2023-06-18 11:49:35.324618 | Ended at : 2023-06-18 11:51:07.508052 and it took 92.183434009552s
Epoch Training Started at : 2023-06-18 11:51:13.146540 | Ended at : 2023-06-18 11:56:09.769544 and it took 296.62300419807434s
Epoch Training Started at : 2023-06-18 11:56:14.998378 | Ended at : 2023-06-18 11:57:07.672110 and it took 52.67373275756836s
Epoch Training Started at : 2023-06-18 11:57:11.616968 | Ended at : 2023-06-18 11:59:30.261952 and it took 138.64498353004456s
Epoch Training Started at : 2023-06-18 11:59:37.669582 | Ended at : 2023-06-18 12:01:53.805255 and it took 136.1356

## Testing the model

### Load test data

In [ ]:
X_test=np.load('X_test.npy')
X_test.shape

In [ ]:
first  = X_test[0]
Topredict = first[1:]
test = first[:-1]

In [ ]:
testing = []
inputLayer = first[0]
# for i in range(len(first)-1):
for i in range(10):

    learned = reconstruction.learnedPrior(inputLayer.reshape(1,256,256,1))
    testing.append(reconstruction.Generate.predict([inputLayer.reshape(1,256,256,1),learned],steps=1))
    inputLayer = testing[-1]

In [ ]:
len(testing)

In [ ]:
n = 10

plt.figure(figsize=(20,4))
for i in range(n):
    # original
    ax = plt.subplot(2,n,i+1)
    plt.imshow(Topredict[i].astype("float32"))
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # reconstructed
    ax = plt.subplot(2,n,i+1+n)
    plt.imshow(testing[i].reshape(256,256,1))
    plt.title('reconstructed')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
# reconstruction.Generate.save('reconstruction_10batch_10epochs.h5')

In [ ]:
file_path = "reconstructed.raw"
testing.insert(0, first[0].reshape(1,256,256,1))
print(len(testing))

In [ ]:
final = np.array(testing,np.float32).reshape(250,256,256)
final.shape

In [ ]:
file_path = "my_array.npy"
np.save("file", final)

In [ ]:
final.tofile(file_path)

In [ ]:
Volume = np.fromfile(file_path, dtype=np.uint8)
Volume = Volume.reshape(250,256,256)
Volume.shape